### About this notebook: 
<font color='grey'>This notebook is used to answer </font><font color='red'>which gene knockout or gene knockdown </font> show sensitivity to <font color='red'> certain gene mutation or the mutation of a group of genes.</font> <br/>

<font color='blue'>The functional screening data and omics data for cell lines is from the Depmap and CCLE project from the Broad institute (DepMap Public 20Q3). To use this jupyter notebook and the data which are used in the jupyter notebook, Please cite the following papers</font> <br/>

....our paper

For this DepMap release:
DepMap, Broad (2020): DepMap 20Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.11791698.v2.

For CRISPR datasets:
Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.

For omics datasets:
Mahmoud Ghandi, Franklin W. Huang, Judit Jané-Valbuena, Gregory V. Kryukov, ... Todd R. Golub, Levi A. Garraway & William R. Sellers. 2019. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019).


In [ ]:
#Make sure to have the libraries required to be installed.
from google.cloud import bigquery
import pandas as pd
import numpy as np
import ipywidgets as widgets
from scipy import stats 
import statsmodels.stats.multitest as multi
import sys
sys.path.append('../scripts/')
import mutation_dependent_PP as mdpp



In [ ]:
# Users need to a google cloud project to query the data in the BigQuery tables. 
project_id='syntheticlethality'
client = bigquery.Client(project_id)

!gcloud auth login

#### Get mutation data from CCLE, crispr gene knockout effects from Depmap and shRNA gene knockdown gene dependency data from demeter

In [ ]:
Mut_mat = mdpp.get_ccle_mutation_data()
Demeter_data = mdpp.get_demeter_shRNA_data()
Depmap_matrix = mdpp.get_depmap_crispr_data()

#### Set user input:
###### 1, Data_source: only two options are avaiable, "shRNA" or "Crispr", datatype: string
###### 2, Mutated genes to be interested. It can be a list of genes or one single gene in a list format. 
###### 3, Tumor types to be included in the analysis. Users can select 'pancancer' or select one or multiple tumor types to theirs interests.

In [ ]:
# User input
Data_source = "shRNA" # only two options are avaiable, "shRNA" or "Crispr", datatype: string
Gene_list = ['BRCA2'] # data type: list

In [ ]:
query = ''' 
SELECT DepMap_ID, primary_disease,TCGA_subtype
FROM `syntheticlethality.DepMap_public_20Q3.sample_info_Depmap_withTCGA_labels` 
'''
sample_info = client.query(query).result().to_dataframe()

pancancer_cls = sample_info.loc[~sample_info['primary_disease'].isin(['Non-Cancerous','Unknown','Engineered','Immortalized'])]
pancancer_cls = pancancer_cls.loc[~(pancancer_cls['primary_disease'].isna())]

TCGA_list = [x for x in list(set(pancancer_cls['TCGA_subtype'])) if x == x]

tumor_type = widgets.SelectMultiple(
    options=['pancancer'] + TCGA_list  ,
    value=[],
    description='Tumor type',
    disabled=False
)
display(tumor_type)

#### Select shRNA dataset or Crispr dataset to infer synthetic lethality pairs for mutated genes! 

In [ ]:
if Data_source == "shRNA"
    result = mdpp.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list, Mut_mat, Demeter_data, Data_source)

In [ ]:
if Data_source == "Crispr"
    result = mdpp.Mutational_based_SL_pipeline(list(tumor_type.value), Gene_list, Mut_mat, Depmap_matrix, Data_source)